In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt

from mne_bids import BIDSPath

import pandas as pd

plt.style.use("figures.mplstyle")

In [2]:
bids_root = "../../monkey"

## stimuli statistics

In [3]:
df = pd.read_csv(f"{bids_root}/stimuli/podcast_transcript.csv")
len(df)

5136

## bad electrodes

In [17]:
channels_tsv = BIDSPath(root=bids_root, datatype='ieeg', suffix='channels', extension='.tsv')
dfs = []
for channel_path in channels_tsv.match():
    df = pd.read_csv(channel_path, sep='\t')
    df.insert(0, 'subject', channel_path.subject)
    dfs.append(df)
df = pd.concat(dfs)
df

,subject,name,type,units,low_cutoff,high_cutoff,sampling_frequency,status,status_description
0,01,G1,ECOG,NaN,NaN,NaN,512.0,bad,no localization
1,01,G2,ECOG,NaN,NaN,NaN,512.0,good,NaN
2,01,G3,ECOG,NaN,NaN,NaN,512.0,good,NaN
3,01,G4,ECOG,NaN,NaN,NaN,512.0,good,NaN
4,01,G5,ECOG,NaN,NaN,NaN,512.0,good,NaN
...,...,...,...,...,...,...,...,...,...
200,09,DC1,MISC,uV,NaN,NaN,512.0,good,NaN
201,09,DC2,MISC,uV,NaN,NaN,512.0,good,NaN
202,09,DC3,MISC,uV,NaN,NaN,512.0,good,NaN
203,09,DC4,MISC,uV,NaN,NaN,512.0,good,NaN


In [19]:
df.groupby('status_description').count()

,subject,name,type,units,low_cutoff,high_cutoff,sampling_frequency,status
status_description,,,,,,,,
no localization,80,80,80,73,0,0,80,80
noisy psd,31,31,31,27,0,0,31,31


## electrode locations

In [20]:
channels_tsv = BIDSPath(root=bids_root, datatype='ieeg', space="MNI152NLin2009aSym", suffix='electrodes', extension='.tsv')

dfs = []
for channel_path in channels_tsv.match():
    df = pd.read_csv(channel_path, sep='\t')
    df['hemi'] = df['x'].apply(lambda x: 'RH' if x > 0 else 'LH')
    df.insert(0, 'subject', channel_path.subject)
    dfs.append(df)
df = pd.concat(dfs)
df

,subject,name,x,y,z,size,group,hemi
0,01,G2,60.000000,-1.333333,-29.333333,4.154756,G,RH
1,01,G3,66.000000,-9.333333,-21.333333,4.154756,G,RH
2,01,G4,69.000000,-16.000000,-14.000000,4.154756,G,RH
3,01,G5,70.666667,-24.666667,-9.000000,4.154756,G,RH
4,01,G6,71.000000,-32.000000,-2.000000,4.154756,G,RH
...,...,...,...,...,...,...,...,...
183,09,DPMT4,-45.600000,-21.800000,-0.800000,4.154756,D,LH
184,09,DPMT5,-50.750000,-22.250000,-1.250000,4.154756,D,LH
185,09,DPMT6,-54.000000,-19.500000,-1.500000,4.154756,D,LH
186,09,DPMT7,-58.285714,-16.000000,0.285714,4.154756,D,LH


In [21]:
sub_hemi = df.groupby(['subject', 'hemi']).name.count()
sub_hemi = pd.pivot(sub_hemi.reset_index(), index='subject', columns='hemi')
sub_hemi

name       
hemi        LH     RH
subject              
01         NaN  103.0
02        95.0    NaN
03       213.0   51.0
04        77.0   78.0
05       159.0    1.0
06       171.0    NaN
07       119.0    NaN
08        75.0    NaN
09       188.0    NaN

In [22]:
sub_elec_group = df.groupby(['subject', 'group']).name.count()
sub_elec_group = pd.pivot(sub_elec_group.reset_index(), index='subject', columns='group')
sub_elec_group

name                   
group       D    EG      G     S
subject                         
01       16.0   NaN   63.0  24.0
02       52.0   NaN    NaN  43.0
03       72.0  64.0   63.0  65.0
04       76.0   NaN    NaN  79.0
05       42.0   NaN  110.0   8.0
06       36.0  64.0   63.0   8.0
07       24.0   NaN   63.0  32.0
08       20.0   NaN   47.0   8.0
09       24.0  64.0   60.0  40.0

In [23]:
pd.concat((sub_hemi, sub_elec_group), axis=1).fillna('n/a')

name                                
            LH     RH     D    EG      G     S
subject                                       
01         n/a  103.0  16.0   n/a   63.0  24.0
02        95.0    n/a  52.0   n/a    n/a  43.0
03       213.0   51.0  72.0  64.0   63.0  65.0
04        77.0   78.0  76.0   n/a    n/a  79.0
05       159.0    1.0  42.0   n/a  110.0   8.0
06       171.0    n/a  36.0  64.0   63.0   8.0
07       119.0    n/a  24.0   n/a   63.0  32.0
08        75.0    n/a  20.0   n/a   47.0   8.0
09       188.0    n/a  24.0  64.0   60.0  40.0